In [1]:
# Install this package if running on your local machine
# !pip install -q tensorflow-datasets

In [2]:
import tensorflow as tf 
print(tf.__version__) 

2.9.2


In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCQR4TH/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCQR4TH/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteCQR4TH/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
# print(info)

### Split the dataset

In [5]:
print(imdb)

{Split('train'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('test'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('unsupervised'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


In [6]:
for example, i in imdb['train'].take(1):
  print(example.numpy().decode('utf8'))
  # print(i)

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.


In [8]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []


for s, l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

for s, l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

### Generate Padded Sequences

In [9]:
vocab_size = 10000
max_length = 120
embeding_dim = 16
trunc_type = 'post'
oov_tok = ''

In [ ]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
word_index
# num_words = len(tokenizer.word_index) + 1
# sequences = tokenizer.texts_to_sequences(training_sentences)
# padded = pad_sequences(sequences, maxlen=max_length)

# testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
# testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [11]:
padded[0, :].shape

(120,)

In [12]:
np.unique(padded)

array([   0,    1,    2, ..., 9997, 9998, 9999], dtype=int32)

### Build and Compile the Model

In [13]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embeding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [14]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.4572 - accuracy: 0.7634 - val_loss: 0.3200 - val_accuracy: 0.8589
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2062 - accuracy: 0.9217 - val_loss: 0.3529 - val_accuracy: 0.8488
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0826 - accuracy: 0.9778 - val_loss: 0.4064 - val_accuracy: 0.8463
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0192 - accuracy: 0.9978 - val_loss: 0.4967 - val_accuracy: 0.8431
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0049 - accuracy: 0.9998 - val_loss: 0.5443 - val_accuracy: 0.8444
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.5881 - val_accuracy: 0.8451
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 7.9136e-04 - accuracy: 1.0000 - val_loss: 0.6294 - val_accuracy: 0.8457
Ep

### Visualize Word Embeddings

In [16]:
embedding_layer = model.layers[0]
embedding_weights = embedding_layer.get_weights()[0]
print(embedding_weights.shape) 

(10000, 16)


In [ ]:
reverse_word_index = tokenizer.index_word
# reverse_word_index


* vecs.tsv - contains the vector weights of each word in the vocabulary
* meta.tsv - contains the words in the vocabulary


In [28]:
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word_name = reverse_word_index[word_num]
  word_embedding = embedding_weights[word_num]
  out_m.write(word_name + "\n")
  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

out_m.close()
out_v.close()

In [29]:
try:
  from google.colab import files
except ImportError:
  pass

else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>